In [1]:
import numpy as np
from numpy import random as r
from itertools import combinations


In [2]:
# Parameters
teams = ["Adelaide Crows", "Brisbane Lions", "Carlton Blues", "Collingwood Magpies",
     "Essendon Bombers", "Fremantle Dockers", "Geelong Cats", "Gold Coast Suns",
     "Greater Western Sydney Giants", "Hawthorn Hawks", "Melbourne Demons",
     "North Melbourne Kangaroos", "Port Adelaide Power", "Richmond Tigers",
     "St Kilda Saints", "Sydney Swans", "West Coast Eagles", "Western Bulldogs"]

team_numbers = {team: number for number, team in enumerate(sorted(teams), start=0)}
print(team_numbers['Carlton Blues'])

locations = ['VIC','NSW','SA','WA','QLD']
location_numbers = {location: number for number, location in enumerate(sorted(locations), start=0)}


home_locations = ['SA','QLD','VIC','VIC','VIC','WA','VIC','QLD','NSW','VIC',
                          'VIC','VIC','SA','VIC','VIC','NSW','WA','VIC']


ranking = [10,2,5,1,11,14,12,15,7,16,4,17,3,13,6,8,18,9] # 2023 pre-finals rankings

team_fans = [60,45,93,102,79,61,81,20,33,72,68,50,60,100,58,63,100,55] # 2023 number of members, in 000's

stadiums = ['MCG','Marvel','GMHBA','Adelaide Oval','Optus','Gabba','HBS','SCG','Giants']
stadium_numbers = {stadium: number for number, stadium in enumerate(sorted(stadiums), start=0)}

stadium_locations = ['VIC','VIC','VIC','SA','WA','QLD','QLD','NSW','NSW']

home_stadiums = [['Adelaide Oval'],['Gabba'],['MCG','Marvel'],['MCG','Marvel'],['MCG','Marvel'],['Optus'],['GMHBA'],
                 ['HBS'],['Giants'],['MCG'],['MCG'],['Marvel'],['Adelaide Oval'],
                 ['MCG'],['Marvel'],['SCG'],['Optus'],['Marvel']]


    
home_location_stadiums = [[] for i in range(len(teams))]
for i in range(len(teams)):
    for j in range(len(stadiums)):
        if stadium_locations[j] == home_locations[i]:
            home_location_stadiums[i].append(stadiums[j])


stadium_size = [100,53,40,54,60,39,27,47,24] # Stadium sizes in 000's


rivals = [['Port Adelaide Power'],['Gold Coast Suns','Collingwood Magpies'], # From wikipedia
          ['Essendon Bombers','Richmond Tigers','Collingwood Magpies', 'Fremantle Dockers'],
          ['Carlton Blues','Essendon Bombers','Brisbane Lions','Melbourne Demons','Richmond Tigers','Geelong Cats',
           'Hawthorn Hawks', 'West Coast Eagles','Port Adelaide Power'],
          ['Carlton Blues','Collingwood Magpies','Richmond Tigers','Hawthorn Hawks','North Melbourne Kangaroos'],
          ['Carlton Blues','West Coast Eagles'],['Collingwood Magpies','Hawthorn Hawks'],['Brisbane Lions'],
          ['Sydney Swans','Western Bulldogs'],['Hawthorn Hawks','Geelong Cats','Essendon Bombers', 'North Melbourne Kangaroos'],
          ['Collingwood Magpies'],['Essendon Bombers','Hawthorn Hawks'],['Adelaide Crows','Collingwood Magpies'],
          ['Carlton Blues','Collingwood Magpies','Essendon Bombers'],['Sydney Swans'],
          ['West Coast Eagles','St Kilda Saints','Greater Western Sydney Giants','Hawthorn Hawks'],
          ['Fremantle Dockers','Sydney Swans','Collingwood Magpies'],['Greater Western Sydney Giants']]

rivals_num = [[team_numbers[i] for i in rivals[j]] for j in range(len(rivals))]


timeslots = [i for i in range(7)]
timeslot_values = [100,130,50,60,110,50,40] # Change later according to attendances
timeslot_names = ['Thursday Night','Friday Night','Saturday Afternoon','Saturday Evening',
                  'Saturday Night','Sunday Afternoon','Sunday Evening']

rounds = [i for i in range(22)]

Ts = range(len(teams))
Ss = range(len(stadiums))
timeslots = range(7)
rounds = range(22)


# Decision Variables
fixture_matrix = [[[[[0 for i in Ts] for j in Ts] for s in Ss] for t in timeslots] for r in rounds]


# Attractiveness Function Parameters, adjust them as needed
alpha = 1.0
beta = 1.0
gamma = 1.0
sigma = 1.0
xi = 1.0

def attractiveness(i, j, s, t, r):
    score = 1
    
    if j in rivals[i]:
        score *= 1+alpha
    
    score /= max(abs(ranking[i]-ranking[j]),1)
    score  /= (ranking[i]+ranking[j])/2
    
    if stadium_locations[s] == home_locations[j]:
        score *= (1+beta)
        
    score *= stadium_size[s]
    score *= (team_fans[i]+team_fans[j])
    
    score *= timeslot_values[t]
    
    return score


2


In [477]:
def simulated_annealing_solve(initial_schedule, cooling_type, starting_temp, final_temp, cooling_size):
    #Establish cooling schedule
    if cooling_type == 'geometric':
        #Geometric cooling for lazy - Set an initial T, a geometric factor, and a size
        initial_T = starting_temp
        decay = (final_temp/starting_temp)**(1/cooling_size)
        cooling_schedule = [initial_T*decay**i for i in range(0,cooling_size)]
        #print(cooling_schedule)

    current_schedule = initial_schedule
    current_objective = objective(current_schedule)
    best_schedule = current_schedule
    best_objective = objective(current_schedule)

    #NB the way of doing simulated annealing in the lecture notes is a bit off, we just loop on T, rather than looping on T and nested loop of certain number iterations at each T
    for T in cooling_schedule:
        #Currently very inefficient - calling the whole objective function each time, rather than having random_neighbourhood return a delta objective value
        new_schedule = random_neighbourhood(current_schedule)
        new_objective = objective(new_schedule)
        
        if new_objective < current_objective or r.random() <= np.exp(-(new_objective-current_objective)/T):
            current_schedule = new_schedule
            current_objective = new_objective
            
            if current_objective < best_objective:
                best_objective = current_objective
                best_schedule = current_schedule

    return best_schedule, best_objective



In [27]:
#Dropping in feasibility and objective functions from GA.ipynb
#Currently updating to use np array and messing with constraints violations and how they're counted

def feasibility(fixture):
    violated = 0
    critical = 0
    
    # Each team plays once a week
    for i in Ts: 
        for r in rounds:
            #See how many matches team i plays in round r
            tally = np.sum(fixture[i,:,:,:,r])+np.sum(fixture[:,i,:,:,r]) 
            if tally != 1:
                critical += 1
            
    # Each team has eleven home games
    for i in Ts:
        tally = np.sum(fixture[i,:,:,:,:])
        if tally != 11:
            critical += 1
    
    # Teams can't play themselves, and play all other teams once or twice (not twice away, or twice home)
    for i in Ts:
        #Can't play self
        tally1 = np.sum(fixture[i,i,:,:,:])
        if tally1 > 0:
            critical += 1 
        
        for j in Ts:
            if i != j:
                # Cannot play two home games against another 
                tally2 = np.sum(fixture[i,j,:,:,:])
                
                # Cannot not play a team
                tally3 = np.sum(fixture[i,j,:,:,:]) + np.sum(fixture[j, i, :, :, :])
            
                #Update number of critical constraint violations
                if tally2 > 1:
                    critical += 1
                if tally3 < 1:
                    critical += 1
    
    #Commenting out for now as I can't parse it
    #At least a five day break 
    #tally = 0        
    #for i in Ts:
    #    for r in rounds[:-1]:
    #        tally = tally + sum(sum(sum(fixture[i][j][s][t][r]+fixture[r][t][s][i][j] for j in Ts) for s in Ss) for t in [5,6]) + sum(sum(sum(fixture[r+1][t][s][j][i]+fixture[r+1][t][s][j][i] for j in Ts) for s in Ss) for t in [0])
    #if 1 < tally:
    #    violated += 1

    # No three games in a row outside home location
    #tally = 0
    #for i in Ts:
    #    for r in rounds[:-2]:
    #        tally += sum(sum(sum(sum(fixture[r_][t][stadium_numbers[s]][j][i]+fixture[r_][t][stadium_numbers[s]][i][j] for j in Ts) for s in home_location_stadiums[i]) for t in timeslots) for r_ in range(r,r+3)) 
    #if tally < 1:
    #    violated += 1 

    # No four away games in a row
    #tally = 0
    #for i in Ts:
    #    for r in rounds[:-3]:
    #        tally += sum(sum(sum(sum(fixture[r_][t][s][j][i] for j in Ts) for s in Ss) for t in timeslots)
    #                                 for r_ in range(r,r+4)) 
    if tally < 1:
        violated += 1  

    # No 2+ games in one day in the same stadium
    for r in rounds:
        tally1 = 0
        tally2 = 0
        tally3 = 0
        for s in Ss:
            # Saturday & Sunday Games 
            tally1 += sum(sum(sum(fixture[i][j][s][t][r] for i in Ts) for j in Ts) for t in [5, 6]) 
            tally2 += sum(sum(sum(fixture[i][j][s][t][r] for i in Ts) for j in Ts) for t in [2, 3, 4])
            # One game per timeslot per stadium 
            for t in [0,1]:
                tally3 += sum(sum(fixture[i][j][s][t][r] for i in Ts) for j in Ts)  
        if tally1 > 1:
            violated += 1
        if tally2 > 1:
            violated += 1
        if tally3 > 1:
            violated += 1
    
    tally1 = 0
    tally2 = 0
    tally3 = 0
    tally4 = 0

    # No more than two games in any timeslot, and only one on Thursday and Friday night, incentivise games in each timeslot
    for r in rounds:
        
        # At least 2 sunday games 
        tally1 += sum(sum(sum(sum(fixture[i][j][s][t][r] for i in Ts) for j in Ts) for s in Ss) for t in [5,6])
        
        # One Thursday & Friday Night Game
        for t in [0,1]:
            tally2 += sum(sum(sum(fixture[i][j][s][t][r] for i in Ts) for j in Ts) for s in Ss)
        
        # No More Than Two Simultaneous Games
        for t in [2,3,4,5,6]:
            tally3 += sum(sum(sum(fixture[i][j][s][t][r] for i in Ts) for j in Ts) for s in Ss)     

        for t in timeslots:
            if sum(sum(sum(fixture[i][j][s][t][r] for i in Ts) for j in Ts) for s in Ss) < 1: 
                tally4 += 1
    
    if tally1 < 2:
        violated += 1
    if tally2 > 1: 
        violated += 1
    if tally3 > 2:
        violated += 1
    if tally4 > 0:
        violated += 1

    return violated, critical

def objective(fixture):

    violated, critical = feasibility(fixture)

    constraint_penalty = 100000000
    critical_penalty = 1000000
    # for i in Ts:
    #     for j in Ts:
    #         for s in Ss:
    #             for r in rounds:
    #                 for t in timeslots:
    #                     print(attractiveness(i,j,s,t,r))

    objective_value = constraint_penalty*violated + critical_penalty*critical + sum(sum(sum(sum(sum(attractiveness(i,j,s,t,r) for i in Ts) for j in Ts) for s in Ss) for t in timeslots) for r in rounds) 

    return objective_value

In [25]:
#A schedule is  a 5d array, where S[i,j,s,t,r] is a boolean that indicates whether team i plays team j in stadium k at timeslot l in round m

def random_neighbour(schedule):
    #Function that gets a random neighbour, choosing which neighbourhood to explore with a particular probability
    #Parameters for tuning likeliness of using a particular neighbourhood function
    a = 0.33
    b = 0.66
    p = r.rand()
    if p<a:
        new_schedule = random_neighbour_home_swap(schedule)
    elif p>=a and p<b:
        new_schedule = random_neighbour_match_move(schedule)
    else:
        new_schedule = random_neighbour_double_swap(schedule)
    
    return new_schedule
    
def random_neighbour_home_swap(schedule):
    #Function that swaps a random match from home to away
    new_schedule = schedule.copy()
    
    #Pick two teams
    i, j = r.choice(range(0,18), 2, replace=False)

    #Choose one of the one plus games they play
    #Aight, so this returns a list of indices, each should be (i,j, non ij index of any actual ij matches)
    i_home = [(i,j) + tuple(index) for index in zip(*np.nonzero(schedule[i,j,:,:,:]))]
    j_home = [(j,i) + tuple(index) for index in zip(*np.nonzero(schedule[j,i,:,:,:]))]
    
    #Concatenate the two lists of indices for i vs j matches
    all_matches = i_home+j_home
    print(all_matches)
    
    #Pick one
    old_match_index = r.randint(len(all_matches))
    old_match = all_matches[old_match_index]
    
    #Flip who plays at home
    new_match = list(old_match)
    new_match[0], new_match[1] = new_match[1], new_match[0]
    new_match = tuple(new_match)
    new_schedule[new_match] = 1
    new_schedule[old_match] = 0
    
    #Return our new schedule
    return new_schedule
    
def random_neighbour_match_move(schedule):
    #Function that moves a random match to a different time
    new_schedule = schedule.copy()
    
    #Pick hometeam
    i = r.randint(18)
    #Gets all the homegames the team plays
    homegames = [[i] + list(index) for index in zip(*np.nonzero(schedule[i,:,:,:,:]))]
    #Pick one
    old_match = homegames[r.randint(len(homegames))]
    
    
    done = False
    while done == False:
        #Pick a new timeslot and round
        t = r.randint(7)
        round = r.randint(22)
        
        #If noone else is playing in this stadium at this time, we move the match here
        if np.sum(schedule[:,:,old_match[2],t,round]) == 0:
            done = True
            new_schedule[old_match[0], old_match[1], old_match[2], t, round] = 1
            new_schedule[tuple(old_match)] = 0
    
    #Return the new schedule
    return new_schedule
    
    
    
def random_neighbour_double_swap(schedule):
    #Function that takes two random double matches and swaps two of the teams between matches
    new_schedule = schedule.copy()
    #Find two pairs of pairs of teams that play twice
    done = False
    while done == False:
        #pick two teams
        i, j, k, l = r.choice(range(0,18), 4, replace=False)
        if np.sum(schedule[i,j,:,:,:])+np.sum(schedule[j,i,:,:,:]) == 2 and np.sum(schedule[k,l,:,:,:]) + np.sum(schedule[l,k,:,:,:])  == 2:
            done = True
    
    #Find all the matches i  and j play together
    i_home = [(i,j) + tuple(index) for index in zip(*np.nonzero(schedule[i,j,:,:,:]))]
    j_home = [(j,i) + tuple(index) for index in zip(*np.nonzero(schedule[j,i,:,:,:]))]
    i_j_matches = i_home + j_home
    
    #Find all the matches k and l play together
    k_home = [(k,l) + tuple(index) for index in zip(*np.nonzero(schedule[k,l,:,:,:]))]
    l_home = [(l,k) + tuple(index) for index in zip(*np.nonzero(schedule[l,k,:,:,:]))]
    k_l_matches = k_home + l_home
    
    #Picks one of the i vs j matches, and one of the k vs l matches
    i_j_to_swap = i_j_matches[r.randint(len(i_j_matches))]
    k_l_to_swap = k_l_matches[r.randint(len(k_l_matches))]
    
    #Gets indices of swapped matches - We're swapping the away teams of the two matches
    i_l_match = (i_j_to_swap[0], l) + i_j_to_swap[2:]
    k_j_match = (k_l_to_swap[0], j) + k_l_to_swap[2:]

    
    #Performs the swap
    new_schedule[i_j_to_swap] = 0
    new_schedule[k_l_to_swap] = 0
    new_schedule[i_l_match] = 1
    new_schedule[k_j_match] = 1
    
    #Return the new schedule
    return new_schedule
    

In [28]:
initial_fixture = np.transpose(np.load('solutions/mip_initial_fixture.npy'),(4, 3, 2, 1, 0))
print('Original objective:', objective(initial_fixture))

IndexError: index 18 is out of bounds for axis 0 with size 18

In [40]:
#Scrap code testing
a = np.array([1,1,0,0,0,0,1,0,1])
print(a)
print(sum(a))

a_ones = np.nonzero(a)[0]
print(a_ones)
i = a_ones[r.randint(len(a_ones))]
print(i)

print(r.choice(np.nonzero(a)[0]))

#Working out how to reorder the indexing of fixture matrix
print(np.shape(fixture_matrix))
new_fixture_matrix = np.transpose(fixture_matrix, (4, 3, 2, 1, 0))
print(np.shape(new_fixture_matrix))
fixture_matrix

#print(type(r.choice(list(range(0,10)),2, replace=False)))
a = [1,2,3,4,5,6,7,8]
i, j = r.choice(range(0,18),2,replace=False)
print(i, j)

c = np.array([[1,0,0],[0,1,1]])
print(c)
print(list(zip(*np.nonzero(c))))


p = (1,2,3)
print([1+i for i in p])
#print(np.concatenate(([i,j],c)))
t = np.zeros((30,1))
print([r.randint(5) for i in t])


q = [1,2,3,4,5]
print(q[0:3])
print(r.rand())

print(list(range(len(teams))))
print(list(rounds))
print(list(rounds[:-1]))

[1 1 0 0 0 0 1 0 1]
4
[0 1 6 8]
8
8
(22, 7, 9, 18, 18)
(18, 18, 9, 7, 22)
14 17
[[1 0 0]
 [0 1 1]]
[(0, 0), (1, 1), (1, 2)]
[2, 3, 4]
[2, 1, 3, 1, 0, 2, 2, 4, 1, 4, 2, 0, 3, 4, 3, 1, 4, 2, 3, 0, 2, 4, 3, 0, 0, 3, 4, 2, 0, 1]
[1, 2, 3]
0.5422091468950377
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
